In [ ]:
# do with multiple url and save it in separate file
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
import time 

#Movie genre links
links=['animation','biography','comedy','crime','documentary','drama','family','fantasy','game-show','history','horror','music','musical','mystery','news','reality-tv','romance','sci-fi','sport','talk-show','thriller','war','western']

#Initialize chrome browser
driver = webdriver.Chrome()

#Loop for extract the data by genre links
for link in links:

    #set default empty for all variables
    titles = []
    ratings = []
    votings = []
    durations = []

    #Navigate to genre-specific IMDb page
    url = F'https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres={link}'
    driver.get(url)
    #Maimize the chrome window size
    driver.maximize_window()


    #x_path of load more button
    load_more_xpath = '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span'
    try:
        #Create a EC for click a clik load more button
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, load_more_xpath)))
        #Find load more button
        load_button = driver.find_element(By.XPATH, load_more_xpath)
        #Click load more button
        ActionChains(driver).move_to_element(load_button).click(load_button).perform()
        time.sleep(5)  # Wait for the new content to load
        print("Clicked 50 more button.")
            
    except Exception as e:
        print(f"No more '50 more' button found: {e}")
        
    def extract_movie_data():
        """Extracts movie data from the page using relative paths."""
        # Get all movie list items
        movie_xpath = '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li'
        data_points = driver.find_elements(By.XPATH, movie_xpath)
        
        print(f"Found {len(data_points)} movies on the page.")
        
        for data in data_points:
            try:
                # extract each box data into separate variables 
                title = data.find_element(By.XPATH, './/div/div/div/div[1]/div[2]/div[1]/a/h3').text
                rating = data.find_element(By.XPATH, './/div/div/div/div[1]/div[2]/span/div/span/span[1]').text
                voting = data.find_element(By.XPATH, './/div/div/div/div[1]/div[2]/span/div/span/span[2]').text
                duration = data.find_element(By.XPATH, './/div/div/div/div[1]/div[2]/div[2]/span[2]').text
                #store all box data into separate variable
                titles.append(title)
                ratings.append(rating)
                votings.append(voting)
                durations.append(duration)

                #store it in different csv file genre vise
                print(f"Apended movie is: {title}")
            except Exception as e:
                print(f"Error extracting data for one movie: {e}")



    # Once all movies are loaded, extract data one final time to catch any missed items.
    extract_movie_data()

    print("Movie scraping is successfully done")
    print(f"Total movies scraped: {len(titles)}")

    table={'titles':titles,'durations':durations,'votings':votings,'ratings':ratings}

    output=pd.DataFrame(table)
    try:
        output.to_csv(fr'C:/Users/Admin/Music/New folder/Exception/genere_{link}.csv')
        print(f"We Save the{link} File")
    except Exception as E:
        print("Saving issue",E)
       
driver.quit()    


    
print('Succesfully finish the scrabing and save the file in desired folder')




In [ ]:
import pandas as pd

# genre names
links=['action', 'adventure' ,'animation','biography','comedy','crime','documentary','drama','family','fantasy','game-show','history','horror','music','musical','mystery','news','reality-tv','romance','sci-fi','sport','talk-show','thriller','war','western']
merged_df=pd.DataFrame()
# using loop clean each csv and store as mergerd csv
for link in links:
    #Read the CSV file
    data=pd.read_csv(Fr"C:/Users/Admin/Music/New folder/Exception/genere_{link}.csv")
    
    df=pd.DataFrame(data)
    
    #Cleaning the movie name , extract the unwanted letters and synbols and space
    df['titles']=df['titles'].str.extract(r'^[^A-Za-z]*(.*)')
    df['titles']=df['titles'].str.strip()

    df.drop(['Unnamed: 0'],axis=1,inplace=True)  #this drop the index column

    #clean hours column and change it to minitues
    df['hours']=df['durations'].str.extract(r'(^[0-9])h',expand=False) # store hour value of duartion
    df['min']=df['durations'].str.extract(r'(^[0-9])m',expand=False)  # store miniutes value of duration

    df['hours']=pd.to_numeric(df['hours'],errors='coerce').fillna(0).astype(int) #Change Data type
    df['min']=pd.to_numeric(df['min'],errors='coerce').fillna(0).astype(int)

    df['duration_min']=(df['hours']*60)+df['min']  # Duration hrs--> miniutes
    # drop old duration column
    df.drop(['hours','min','durations'],axis=1,inplace=True)

    #Clean votings column , remove unwanted string
    df['votings'] = df['votings'].astype(str).str.replace(r"[()\-\s]", "", regex=True).str.lower()

    # Extract numeric part and convert to float
    df['num'] = df['votings'].str.extract(r'([\d.]+)')[0].astype(float)

    # Check if 'k' is present and multiply by 1000 if needed
    df['votings'] = (df['num'] * df['votings'].str.contains('k', na=False).replace({True: 1000, False: 1})).fillna(0).astype(int).infer_objects(copy=False)
    df.drop(['num'],axis=1,inplace=True)

    

    df['genere']=link #fill the genre name column
    merged_df = pd.concat([merged_df, df], ignore_index=True) # Merge the data
    # df.to_csv(r'C:/Users/Admin/Music/Clean_csv/mergedcsv',mode='a', index=False, header=not bool(link == links[0]))

#Add MYMDb score column
C = merged_df['ratings'].mean()  # Global Mean Rating
M = merged_df['votings'].quantile(0.90)  # Minimum Votes Threshold (90th percentile)



 #  Bayesian Average formule
    # score = (V*R)+(M*C)/V+M
#  Understanding the Formula  
# V (Voting Count): Number of votes received by the movie.
# R (Rating): Average rating of the movie.
# M (Minimum Votes): A threshold value (e.g., only consider movies with at least 1000 votes).
# C (Mean Rating Across All Movies): The global average rating of all movies in the dataset.

# Apply Bayesian Average Formula
merged_df['score'] = ((merged_df['votings'] * merged_df['ratings']) + (M * C)) / (merged_df['votings'] + M)

# Save Final Merged and Cleaned Data
merged_df.to_csv(r'C:/Users/Admin/Music/Clean_csv/final_Output.csv', index=False)

In [ ]:
import pandas as pd

data=pd.read_csv(r"C:/Users/Admin/Music/Clean_csv/final_output.csv")
df=pd.DataFrame(data)
#add one value for null value
value=df.fillna("September 5")
print(value.isnull().sum())
#save the cleaned file into diffrent file
value.to_csv(r"C:/Users/Admin/Music/Clean_csv/mymdb.csv",index=False) # this the point final_output change to mymdb

In [ ]:
import pandas as pd

data=pd.read_csv(r"C:/Users/Admin/Music/Clean_csv/mymdb.csv")
df=pd.DataFrame(data)
#round the score column digit
df['score']=df['score'].round(2)
#Captilize the genre_name
df['genere']=df["genere"].str.capitalize()
df.to_csv(r"C:/Users/Admin/Music/Clean_csv/mymdb.csv",index=False)
df

In [ ]:
import pandas as pd

# Create genre mapping dictionary , insert genre_id for genre_name
genre_mapping = {
    'Action': 1, 'Adventure': 2, 'Animation': 3, 'Biography': 4, 'Comedy': 5,
    'Crime': 6, 'Documentary': 7, 'Drama': 8, 'Family': 9, 'Fantasy': 10,
    'Game-Show': 11, 'History': 12, 'Horror': 13, 'Music': 14, 'Musical': 15,
    'Mystery': 16, 'News': 17, 'Reality-TV': 18, 'Romance': 19, 'Sci-Fi': 20,
    'Sport': 21, 'Talk-Show': 22, 'Thriller': 23, 'War': 24, 'Western': 25
}

# Read CSV file
df = pd.read_csv('C:/Users/Admin/Music/Clean_csv/mymdb.csv')
print(df.isna().sum())

# Clean and process 'genere' column
df['genere'] = df['genere'].astype(str).str.strip().str.title()  # Standardize text


# Map genres to numerical IDs
df['genre'] = df['genere'].map(genre_mapping)

# Check if any genres are still missing
missing_genres = df[df['genre'].isna()]
print("Missing genres:", missing_genres['genere'].unique())

# Replace NaN values with a default value (e.g., 0 for unknown genres)
df['genre'] = df['genre'].fillna(18).astype(int)

# Save modified dataframe
df.to_csv('C:/Users/Admin/Music/Clean_csv/mymdb2.csv', index=False)

# Verify changes
print("last 5 rows after replacement:")
print(df.tail())


In [ ]:
import pandas as pd
import pymysql

# Load data
data = pd.read_csv(r"C:/Users/Admin/Music/Clean_csv/mymdb2.csv")
df = pd.DataFrame(data)

try:
    # Connect to the database
    connection = pymysql.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="4V44XYoMA7okY9v.root",
        password="IPDxDlc6fXH0d2G0",
        database="mymdb",
        ssl_verify_cert=True,
        ssl_verify_identity=True,
    )
    print("Connected to server")

    cursor = connection.cursor()

    # Ensure column names match the database schema
    insert_query = """INSERT INTO movie_list (movie_name, ratings, votings, genre_id, score, duration) 
                      VALUES (%s, %s, %s, %s, %s, %s)"""

    # Convert NaN values to None (NULL in SQL)
    df = df[['titles', 'ratings', 'votings', 'genre', 'score', 'duration_min']].where(pd.notna(df), None)

    # Convert DataFrame to list of tuples
    values = list(df.itertuples(index=False, name=None))

    # Execute batch insert
    cursor.executemany(insert_query, values)
    connection.commit()
    print("Values added successfully")

except pymysql.MySQLError as error:
    print("Error:", error)

finally: #Close the connection
    if 'connection' in locals() and connection.open:
        cursor.close()
        connection.close()
        print("Connection closed successfully!")
